# DAY-AHEAD BASELINE MODEL

## 2025 ENTSO-E data BZN|NL

In [24]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

repo_root = Path("..").resolve()
sys.path.append(str(repo_root))


#Load 2025 ENTSO-E data
from Source.load_data import load_2025

df_market = load_2025()
df_market = df_market.T.drop_duplicates().T
df_market.head()
df_market.tail()

,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),Total Generation Forecast (MW),Total Actual Generation (MW),MTU (CET/CEST),Actual Total Load (MW),Day-ahead Total Load Forecast (MW),Day-ahead offshore wind (MW),Actual offshore wind (MW),Day-ahead onshore wind (MW),Actual onshore wind (MW),Day-ahead solar (MW),Actual solar (MW)
35035,31/12/2025 22:45:00 - 31/12/2025 23:00:00,BZN|NL,Without Sequence,73.76,10688.22,8141.58,31/12/2025 22:45 - 31/12/2025 23:00,10602.18,11600.78,1057,4027.5,2109,808.08,0,0.0
35036,31/12/2025 23:00:00 - 31/12/2025 23:15:00,BZN|NL,Without Sequence,87.16,10029.89,8164.07,31/12/2025 23:00 - 31/12/2025 23:15,10617.95,11397.23,1088,4076.18,2155,810.64,0,0.0
35037,31/12/2025 23:15:00 - 31/12/2025 23:30:00,BZN|NL,Without Sequence,74.27,9800.48,8158.97,31/12/2025 23:15 - 31/12/2025 23:30,10665.56,11343.97,1116,4116.6,2204,794.55,0,0.0
35038,31/12/2025 23:30:00 - 31/12/2025 23:45:00,BZN|NL,Without Sequence,67.6,9716.1,8056.36,31/12/2025 23:30 - 31/12/2025 23:45,10590.58,11094.52,1140,4069.39,2254,790.05,0,0.0
35039,31/12/2025 23:45:00 - 01/01/2026 00:00:00,BZN|NL,Without Sequence,67.6,9539.61,8044.8,31/12/2025 23:45 - 01/01/2026 00:00,10433.0,10680.22,1164,4209.28,2303,856.55,0,0.0
